In [1]:
import pandas as pd
import numpy as np

# dataset init


In [2]:
train_df = pd.read_csv("train.csv")
train_df = train_df.drop(labels='id', axis='columns')
test_df = pd.read_csv("test.csv")
train_df.shape



(136429, 13)

# data analyzation

In [3]:
train_df.describe()

for cols in train_df.columns:
    print(cols, train_df[cols].isnull().sum())

Product ID 0
Type 0
Air temperature [K] 0
Process temperature [K] 0
Rotational speed [rpm] 0
Torque [Nm] 0
Tool wear [min] 0
Machine failure 0
TWF 0
HDF 0
PWF 0
OSF 0
RNF 0


# model building


In [4]:
features = []

for feature in test_df.columns:
    if feature != 'id':
        features.append(feature)


from sklearn.preprocessing import LabelEncoder

submission = test_df[['id']]
le = LabelEncoder()
for col in test_df.select_dtypes(include=['object']).columns:
    train_df[col] = le.fit_transform(train_df[col])
    test_df[col] = le.fit_transform(test_df[col])

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(train_df[features], train_df['Machine failure'])

RandomForestClassifier()

# neural network testing 

In [7]:
from tensorflow import keras

model = keras.models.Sequential([
    keras.layers.Flatten(input_dim = len(features)),
    keras.layers.Dense(128, activation = 'relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop')
model.fit(train_df[features], train_df['Machine failure'], epochs=5)

Epoch 1/5


ValueError: in user code:

    File "c:\Users\coder\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\coder\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\coder\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\coder\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1051, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\coder\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1109, in compute_loss
        return self.compiled_loss(
    File "c:\Users\coder\AppDa

# predicition

In [ ]:
Y_pred = clf.predict(test_df[features])

# submission

In [ ]:
submission['Machine failure'] = Y_pred
for index, value in enumerate(submission['Machine failure']):
    if value == 1:
        submission['Machine failure'][index] = 0.5
        
print(submission['Machine failure'].unique())

submission.to_csv("submission.csv", index=False)

C:\Users\coder\AppData\Local\Temp\ipykernel_7992\437260501.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission['Machine failure'] = Y_pred
C:\Users\coder\AppData\Local\Temp\ipykernel_7992\437260501.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission['Machine failure'][index] = 0.5


[0.  0.5]
